# テスト実施

In [1]:
# external dependency
import sys
sys.path.append('../')
import nbimporter

from Designed_Primary_Energy_Consumption import get_E_V, Ventilation, LS

import pprint
import pandas as pd
import nbimporter

Importing Jupyter notebook from Designed_Primary_Energy_Consumption.ipynb
Importing Jupyter notebook from ..\_05_Ventilation_System\Ventilation.ipynb
Importing Jupyter notebook from ..\_11_Others\Life_Schedule.ipynb


### テストケース準備

まずはテストケースを読み取ります。

In [2]:
V_A = pd.read_csv('V_A.txt', dtype=object)
V_A.head()

,testcase,bsc_nobeyuka_menseki,vn_houshiki,vn_duct_houhou,vn_duct_method,vn_duct_hishohidenryoku,vn_other_houhou,vn_other_hishohidenryoku,vn_freq,vn_e,hv_houshiki,hv_eff,hv_bal,hv_leak,E_V
0,換気A1,120.08,1,1,-,-,-,-,1,1,2,-,-,-,7293.6905
1,換気A2,120.08,1,2,1,-,-,-,1,1,2,-,-,-,5260.9133
2,換気A3,120.08,1,2,2,-,-,-,1,1,2,-,-,-,3600.8118
3,換気A5,120.08,1,3,-,2.00,-,-,1,1,2,-,-,-,27621.4631
4,換気A6,120.08,1,3,-,0.10,-,-,1,1,2,-,-,-,1872.9512


### テストケースの変換

テストケース表から計算条件を作成する関数を定義します。

In [3]:
def testCase2Data(row):

    DataVentilation = {
        'General' : {        
            'AllFloorArea' : row['bsc_nobeyuka_menseki']
        }
    }
    
    #換気設備の方式
    dicVentilationType = {'1':'DuctType1', '2':'DuctType2', '3':'WallType1', '4':'WallType2'}
    DataVentilation['Ventilation'] = {
            'VentilationType': dicVentilationType[row['vn_houshiki']],
    }
    
    #ダクト式
    if row['vn_houshiki'] == '1' or row['vn_houshiki'] == '2':

        DataVentilation['Ventilation']['DuctTypeSpec'] = {'EnergySavingType': 'None'}
        #特に省エネルギー対策をしていない
        if row['vn_duct_houhou'] == '1':
            DataVentilation['Ventilation']['DuctTypeSpec'] = {'EnergySavingType': 'None'}

        #採用した省エネルギー手法を選択する
        elif row['vn_duct_houhou'] == '2':
            dicEnergySavingMethod = {'1': 'LargeDuct', '2':'LargeDuctAndDCMoter'}
            DataVentilation['Ventilation']['DuctTypeSpec'] = {'EnergySavingType': 'Specification'}
            DataVentilation['Ventilation']['DuctTypeSpec']['EnergySavingMethod'] = dicEnergySavingMethod[row['vn_duct_method']]

        #比消費電力を入力することにより省エネルギー効果を評価する
        elif row['vn_duct_houhou'] == '3':
            DataVentilation['Ventilation']['DuctTypeSpec'] = {'EnergySavingType': 'SFP'}
            DataVentilation['Ventilation']['DuctTypeSpec']['SFP'] = row['vn_duct_hishohidenryoku']
    
    #壁付け式
    elif row['vn_houshiki'] == '3' or row['vn_houshiki'] == '4':
        
        #特に省エネルギー対策をしていない
        if row['vn_other_houhou'] == '1':
            DataVentilation['Ventilation']['WallMountedTypeSpec'] = {
                'EnergySavingType': 'None'
            }
        
        #省エネルギー効果を評価するため比消費電力を入力する
        elif row['vn_other_houhou'] == '2':
            DataVentilation['Ventilation']['WallMountedTypeSpec'] = {
                'EnergySavingType': 'SFP',
                'SFP'             : row['vn_other_hishohidenryoku']
            }
    
    #換気回数
    dicVentilationRate = {'1':'0.5', '2':'0.7', '3':'0.0'}
    DataVentilation['VentilationRate'] = dicVentilationRate[row['vn_freq']]
    
    #第一種換気設備の場合における有効換気量率
    if row['vn_houshiki'] == '1' or row['vn_houshiki'] == '3':
        DataVentilation['EffectiveVentilationRate'] = row['vn_e']
    else:
        DataVentilation['EffectiveVentilationRate'] = 1.0

    #熱交換型換気
    if row['hv_houshiki'] == '1':
        DataVentilation['HeatExchanger'] = {
            'Apply'                   : 'Apply',
            'TemperatureExchangeRate' : row['hv_eff'],
            'BalanceCorrectCoeff'     : row['hv_bal'],
            'AirLeakCorrectCoeff'     : row['hv_leak']
        }
    elif row['hv_houshiki'] == '2':
        DataVentilation['HeatExchanger'] = {'Apply': 'NotApply'}
    
    return DataVentilation

### 動作確認

testCase2Data関数にテストケース1つ分を入力し変換されたデータが得られることを確認します。

In [4]:
DataVentilation = testCase2Data({
    'bsc_nobeyuka_menseki' : '120.8',
    'vn_houshiki'          : '1',
    'vn_duct_houhou'       : '1',
    'vn_freq'              : '1',
    'vn_e'                 : '1',
    'hv_houshiki'          : '2'
})

pprint.pprint(DataVentilation)

{'EffectiveVentilationRate': '1',
 'General': {'AllFloorArea': '120.8'},
 'HeatExchanger': {'Apply': 'NotApply'},
 'Ventilation': {'DuctTypeSpec': {'EnergySavingType': 'None'},
                 'VentilationType': 'DuctType1'},
 'VentilationRate': '0.5'}


### テストケースの実行

テストケースを実行します。

In [5]:
for index, row in V_A.iterrows():
    
    DataVentilation = testCase2Data(row)
    
    print ('{0}'.format(row['testcase']))
    print ('EXPECTED = {0}'.format(row['E_V']))

    _E_E_V = [ [ Ventilation.getHoerlyElectricPower(DataVentilation, day, hour) for hour in range(24) ] for day in LS.get_Annual_Days()]
    _E_V = get_E_V(_E_E_V, 9760)
    print ('RESULT = {0}'.format(_E_V))
    if abs(_E_V - float(row['E_V'])) < 0.01:
        print ('OK')
    else:
        print ('NG')


換気A1
EXPECTED = 7293.6905
RESULT = 7293.690534080002
OK
換気A2
EXPECTED = 5260.9133
RESULT = 5260.913276095999
OK
換気A3
EXPECTED = 3600.8118
RESULT = 3600.8118487424
OK
換気A5
EXPECTED = 27621.4631
RESULT = 27621.46311392
OK
換気A6
EXPECTED = 1872.9512
RESULT = 1872.9511794560005
OK
換気A7
EXPECTED = 653.2848
RESULT = 653.2848246656001
OK
換気A10
EXPECTED = 10004.0602113923
RESULT = 10004.060211392003
OK
換気A11
EXPECTED = 517.766340800038
RESULT = 517.7663408
OK
換気A12
EXPECTED = 5938.50569542396
RESULT = 5938.505695424001
OK
換気A13
EXPECTED = 3770.20995357461
RESULT = 3770.2099535744
OK
換気A14
EXPECTED = 2469.23250846477
RESULT = 2469.2325084646404
OK
換気A16
EXPECTED = 27621.4631139176
RESULT = 27621.46311392
OK
換気A17
EXPECTED = 1872.95117945623
RESULT = 1872.9511794560005
OK
換気A18
EXPECTED = 653.284824665556
RESULT = 653.2848246656001
OK
換気A21
EXPECTED = 5938.50569542396
RESULT = 5938.505695424001
OK
換気A23
EXPECTED = 27621.4631139176
RESULT = 27621.46311392
OK
換気A24
EXPECTED = 1872.95117945623
RESUL